## 09_Integrated_HR_Dashboard.ipynb

**Purpose:**  
Create consolidated datasets for HR dashboards and reporting.

**Input:**  
- `data/processed/`

**Output:**  
- `data/processed/HRMS_dashboard_master.csv`  
- `data/processed/Candidates_dashboard_master.csv`  
- `data/processed/Jobs_dashboard_master.csv`

**Notes:**  
- Integrates outputs from multiple pipelines.  
- Serves as the main data source for BI tools and dashboards.

In [ ]:
import pandas as pd
from pathlib import Path

DATA_PATH = Path(r"C:\Users\abanu\Documents\t_iq_hr\data\processed")
OUT_PATH = DATA_PATH


In [19]:
def clean_columns(df):
    """
    Standardize dataframe columns:
    - strip spaces
    - lowercase
    - replace spaces with underscores
    """
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    return df


In [20]:
# Load
hrms_attrition = pd.read_csv(DATA_PATH / "HRMS_with_attrition_predictions.csv")
hrms_perf = pd.read_csv(DATA_PATH / "HRMS_with_performance_predictions.csv")

# Clean columns
hrms_attrition = clean_columns(hrms_attrition)
hrms_perf = clean_columns(hrms_perf)

# Standardize merge key
hrms_attrition.rename(columns={'employeeid':'employee_id'}, inplace=True)
hrms_perf.rename(columns={'employeeid':'employee_id'}, inplace=True)

# Drop duplicate columns from performance dataframe to avoid _x/_y suffixes
drop_cols = [c for c in hrms_perf.columns if c in hrms_attrition.columns and c != 'employee_id']
hrms_perf_cleaned = hrms_perf.drop(columns=drop_cols, errors='ignore')

# Merge HRMS tables
hrms_master = hrms_attrition.merge(hrms_perf_cleaned, on='employee_id', how='left')

print("HRMS master shape:", hrms_master.shape)
hrms_master.head(3)


HRMS master shape: (10000, 16)


,employee_id,name,department,job_role,location,current_salary,satisfaction_score,engagement_score,num_skills,years_at_company,trainings_count,attrition_probability,attrition_risk,attrition_risk_level,performance_score,performance_band
0,EMP0001,Vikram Singh,HR,Data Scientist,"New York, USA",4544478,0.78,0.80,7,12,0,0.711720,1,Medium,0,Low
1,EMP0002,Karan Patel,Marketing,Data Scientist,"Chennai, India",5180268,0.71,0.93,8,7,4,0.194277,0,Low,0,Low
2,EMP0003,Vikram Malhotra,Marketing,Senior Software Engineer,"Chennai, India",2589268,0.81,0.56,6,3,3,0.554748,1,Medium,0,Low


In [21]:
candidates = pd.read_csv(DATA_PATH / "Candidates_features.csv")
resume_features = pd.read_csv(DATA_PATH / "resume_features.csv")

# Clean columns
candidates = clean_columns(candidates)
resume_features = clean_columns(resume_features)

# Standardize merge key
candidates.rename(columns={'candidateid':'candidate_id'}, inplace=True)
resume_features.rename(columns={'candidateid':'candidate_id'}, inplace=True)

# Drop duplicate columns from resume_features to avoid _x/_y suffixes
drop_cols = [c for c in resume_features.columns if c in candidates.columns and c != 'candidate_id']
resume_cleaned = resume_features.drop(columns=drop_cols, errors='ignore')

# Merge candidate tables
candidate_master = candidates.merge(resume_cleaned, on='candidate_id', how='left')

# Optional: rename experience column for clarity
if 'experience_years_x' in candidate_master.columns:
    candidate_master.rename(columns={'experience_years_x':'experience_years'}, inplace=True)
if 'num_skills_x' in candidate_master.columns:
    candidate_master.rename(columns={'num_skills_x':'num_skills'}, inplace=True)

print("Candidate master shape:", candidate_master.shape)
candidate_master.head(3)


Candidate master shape: (2481, 15)


,candidate_id,resume_str,resume_html,category,resume_len,resume_clean,num_words,resume_word_count,resume_sent_count,category_enc,num_sentences,cleaned_resume,skills,num_skills,experience_years
0,CAND00001,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR,5442,hr administratormarketing associate hr adminis...,674,674.0,26,25,26,hr administratormarketing associate hr adminis...,"['hr', 'payroll']",2,15
1,CAND00002,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR,5572,hr specialist us hr operations summary versati...,708,708.0,26,25,26,hr specialist us hr operations summary versati...,"['hr', 'recruitment']",2,0
2,CAND00003,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR,7720,hr director summary over 20 years experience i...,1017,1017.0,40,25,40,hr director summary over 20 years experience i...,"['excel', 'hr', 'payroll', 'recruitment']",4,20


In [22]:
jobs_cleaned = pd.read_csv(DATA_PATH / "Jobs_cleaned.csv")
jobs_analytics = pd.read_csv(DATA_PATH / "jobs_analytics_features.csv")

# Clean columns
jobs_cleaned = clean_columns(jobs_cleaned)
jobs_analytics = clean_columns(jobs_analytics)

# Standardize merge key
jobs_cleaned.rename(columns={'job_id':'job_id'}, inplace=True)
jobs_analytics.rename(columns={'job_id':'job_id'}, inplace=True)

# Drop duplicates from analytics to avoid _x/_y suffixes
drop_cols = [c for c in jobs_analytics.columns if c in jobs_cleaned.columns and c != 'job_id']
jobs_analytics_cleaned = jobs_analytics.drop(columns=drop_cols, errors='ignore')

# Merge
jobs_master = jobs_cleaned.merge(jobs_analytics_cleaned, on='job_id', how='left')

print("Jobs master shape:", jobs_master.shape)
jobs_master.head(3)


Jobs master shape: (10000, 18)


,job_id,job_title,company,location,industry,seniority,employment_type,salary_min,salary_max,years_experience,skills,job_text,job_len,salary_mid,salary_range,exp_bucket,num_skills_required,job_len_bucket
0,JOB000001,3,122,0,2,3,3,107397,140048,0,Quantitative Analysis;Azure;Feature Engineerin...,"Design, develop and maintain scalable Kubernet...",59,123722.5,32651,Entry,5,Medium
1,JOB000002,13,111,3,6,1,0,42676,71202,5,Spark;Node.js;spaCy;Azure;Kubernetes;GANs,Maintain documentation and operate within Agil...,56,56939.0,28526,Mid,6,Medium
2,JOB000003,26,155,3,2,0,3,117841,140337,10,AWS;GANs;Kotlin;RESTful Design;Tableau;RoBERTa...,Collaborate with cross-functional teams to def...,64,129089.0,22496,Senior,10,Long


In [23]:
# Use department column from HRMS master (original attrition table)
dept_col = 'department'
if dept_col in hrms_master.columns:
    dept_summary = hrms_master.groupby(dept_col).agg(
        total_employees=('employee_id', 'count'),
        high_risk_attrition=('attrition_risk_level', lambda x: (x=='High').sum()),
        high_performance=('performance_band', lambda x: (x=='High').sum())
    ).reset_index()
    dept_summary.rename(columns={dept_col:'department'}, inplace=True)
    print(dept_summary.head(3))
else:
    print(f"Column '{dept_col}' not found in HRMS master.")


     department  total_employees  high_risk_attrition  high_performance
0  Data Science             1033                  128                 0
1   Engineering              965                  244                 0
2       Finance              985                  154                 0


In [24]:
# Flatten skills list
if 'skills' in candidate_master.columns:
    candidate_master['skills_str'] = candidate_master['skills'].apply(
        lambda x: ', '.join(eval(x)) if pd.notna(x) else ''
    )
else:
    candidate_master['skills_str'] = ''

# Preview
cols_to_show = ['candidate_id', 'skills_str', 'experience_years'] if 'experience_years' in candidate_master.columns else ['candidate_id', 'skills_str']
candidate_master[cols_to_show].head(5)


,candidate_id,skills_str,experience_years
0,CAND00001,"hr, payroll",15
1,CAND00002,"hr, recruitment",0
2,CAND00003,"excel, hr, payroll, recruitment",20
3,CAND00004,"excel, hr, payroll",20
4,CAND00005,"excel, hr, payroll, recruitment",0


In [25]:
# HRMS master
hrms_master.to_csv(OUT_PATH / "HRMS_dashboard_master.csv", index=False)

# Candidate master
candidate_master.to_csv(OUT_PATH / "Candidates_dashboard_master.csv", index=False)

# Jobs master
jobs_master.to_csv(OUT_PATH / "Jobs_dashboard_master.csv", index=False)

print("✅ Notebook-09 completed: All dashboard-ready datasets saved")


✅ Notebook-09 completed: All dashboard-ready datasets saved
